In [1]:
import cntk; print(cntk.__version__)

2.7


In [3]:
pwd

'/data/home/cubean/notebooks/git/ImageSimilarityUsingCntk'

In [2]:
cd /data/home/cubean/notebooks/git/ImageSimilarityUsingCntk

/data/home/cubean/notebooks/git/ImageSimilarityUsingCntk


In [4]:
# -*- coding: utf-8 -*-
from helpers import *
locals().update(importlib.import_module("PARAMETERS").__dict__)


####################################
# Main
####################################
random.seed(0)
makeDirectory(procDir)
imgFilenamesTest  = dict()
imgFilenamesTrain = dict()

print("Split images into train or test...")
subdirs = getDirectoriesInDirectory(imgDir)
for subdir in subdirs:
    filenames = getFilesInDirectory(imgDir + subdir, ".jpg")

    # Randomly assign images into train or test
    if imagesSplitBy == 'filename':
        filenames  = randomizeList(filenames)
        splitIndex = int(ratioTrainTest * len(filenames))
        imgFilenamesTrain[subdir] = filenames[:splitIndex]
        imgFilenamesTest[subdir]  = filenames[splitIndex:]

    # Randomly assign whole subdirectories to train or test
    elif imagesSplitBy == 'subdir':
        if random.random() < ratioTrainTest:
            imgFilenamesTrain[subdir] = filenames
        else:
            imgFilenamesTest[subdir]  = filenames
    else:
        raise Exception("Variable 'imagesSplitBy' has to be either 'filename' or 'subdir'")

    # Debug print
    if subdir in imgFilenamesTrain:
        print("Training: {:5} images in directory {}".format(len(imgFilenamesTrain[subdir]), subdir))
    if subdir in imgFilenamesTest:
        print("Testing:  {:5} images in directory {}".format(len(imgFilenamesTest[subdir]), subdir))

# Save assignments of images to train or test
saveToPickle(imgFilenamesTrainPath, imgFilenamesTrain)
saveToPickle(imgFilenamesTestPath,  imgFilenamesTest)

# Mappings label <-> id
lutId2Label = dict()
lutLabel2Id = dict()
for index, key in enumerate(imgFilenamesTrain.keys()):
    lutLabel2Id[key] = index
    lutId2Label[index] = key
saveToPickle(lutLabel2IdPath, lutLabel2Id)
saveToPickle(lutId2LabelPath, lutId2Label)

# Compute positive and negative image pairs
print("Generate training data ...")
imgInfosTrain = getImagePairs(imgFilenamesTrain, train_maxQueryImgsPerSubdir, train_maxNegImgsPerQueryImg)
saveToPickle(imgInfosTrainPath, imgInfosTrain)
print("Generate test data ...")
imgInfosTest = getImagePairs(imgFilenamesTest, test_maxQueryImgsPerSubdir, test_maxNegImgsPerQueryImg)
saveToPickle(imgInfosTestPath, imgInfosTest)

# Sanity check - make sure the test and training set have no images in common
if True:
    print("Verifying if training and test set are disjoint:")
    pathsTest  = getImgPaths(loadFromPickle(imgInfosTestPath))
    pathsTrain = getImgPaths(loadFromPickle(imgInfosTrainPath))

    # Make sure the training set and test set have zero overlap
    overlap = len(pathsTrain.intersection(pathsTest))
    if overlap == 0:
        print("   Check passed: Training and test set share no images.")
    else:
        raise Exception("Training and test set share %d images." % overlap)
print("DONE.")


PARAMETERS: datasetName = costumebox/training/womens-costumes
Split images into train or test...
Training:    75 images in directory womens-elegant-costumes
Testing:     25 images in directory womens-elegant-costumes
Training:    75 images in directory sexy-costumes
Testing:     25 images in directory sexy-costumes
Training:    12 images in directory party-busters
Testing:      4 images in directory party-busters
Training:    75 images in directory womens-plus-size-costumes
Testing:     25 images in directory womens-plus-size-costumes
Generate training data ...
Generated image pairs for 237 query images, each with 1 positive image pair and 100 negative image pairs.
Generate test data ...
Generated image pairs for 64 query images, each with 1 positive image pair and 100 negative image pairs.
Verifying if training and test set are disjoint:
   Check passed: Training and test set share no images.
DONE.


In [5]:
# -*- coding: utf-8 -*-
from helpers import *
from helpers_cntk import *
locals().update(importlib.import_module("PARAMETERS").__dict__)


################################################
# MAIN
################################################
makeDirectory(workingDir)

# Load data
lutLabel2Id = loadFromPickle(lutLabel2IdPath)
imgFilenamesTest  = loadFromPickle(imgFilenamesTestPath)
imgFilenamesTrain = loadFromPickle(imgFilenamesTrainPath)

# Generate cntk test and train data, i.e. (image, label) pairs and write
# them to disk since in-memory passing is currently not supported by cntk
dataTest  = getImgLabelMap(imgFilenamesTest,  imgDir, lutLabel2Id)
dataTrain = getImgLabelMap(imgFilenamesTrain, imgDir, lutLabel2Id)
if rf_boBalanceTrainingSet:
    dataTrain = balanceDatasetUsingDuplicates(dataTrain)
writeTable(cntkTrainMapPath, dataTrain)
writeTable(cntkTestMapPath, dataTest)

# Train model
printDeviceType()
model = train_model(cntkPretrainedModelPath, cntkTrainMapPath, cntkTestMapPath, rf_inputResoluton,
                    rf_maxEpochs, rf_mbSize, rf_maxTrainImages, rf_lrPerMb, rf_momentumPerMb, rf_l2RegWeight,
                    rf_dropoutRate, rf_boFreezeWeights)
model.save(cntkRefinedModelPath)
print("Stored trained model at %s" % cntkRefinedModelPath)

print("DONE. Showing DNN accuracy vs training epoch plot.")
plt.show() # Accuracy vs training epochs plt

ValueError: Cannot load the model /data/home/cubean/notebooks/git/ImageSimilarityUsingCntk/cntk/resources/cntk/ResNet_18.model that is neither a file nor a byte buffer.